In [204]:
import pandas as pd
import sqlite3
import psycopg2

In [205]:
df = pd.read_csv('titanic.csv')

In [206]:
df['Survived'] = df['Survived'].replace({0:'no', 1:'yes'})
df['Pclass'] = df['Pclass'].replace({1:'first',2:'second',3:'third'})
df['Name'] = df['Name'].str.replace(r"[\"\',]", '')

In [207]:
sl_conn = sqlite3.connect('titanic.sqlite3')
df.to_sql(name='passengers', con=sl_conn, if_exists='replace')
sl_curs = sl_conn.cursor()

In [208]:
people = sl_curs.execute('SELECT * FROM passengers;').fetchall()

In [209]:
df.head(2)

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,no,third,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,yes,first,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833


In [210]:
str(people[1])

"(1, 'yes', 'first', 'Mrs. John Bradley (Florence Briggs Thayer) Cumings', 'female', 38.0, 1, 0, 71.2833)"

In [211]:
# Must Replace Placeholders to Run
dbname = 'NAME'
user = 'NAME'
password = 'PASSWORD'
host = 'raja.db.elephantsql.com'

In [212]:
pg_conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
pg_curs = pg_conn.cursor()

In [213]:
pg_curs.execute("CREATE TYPE gender AS ENUM ('male', 'female');")
pg_curs.execute("CREATE TYPE class AS ENUM ('first', 'second', 'third');")
pg_curs.execute("CREATE TYPE survive AS ENUM ('no', 'yes');")

In [214]:
# Scheme for Postgres

create_passenger_table = """
CREATE TABLE passengers_titanic (
    passenger_id SERIAL PRIMARY KEY,
    survived survive,
    pclass class,
    name text,
    sex gender,
    age DOUBLE PRECISION,
    sibl_spouse_aboard INT,
    parent_child_aboard INT,
    fare DOUBLE PRECISION
)
"""

In [215]:
pg_curs.execute(create_passenger_table)

In [216]:
for person in people:
    insert_person = """
    INSERT INTO passengers_titanic
    (survived, pclass, name, sex, age, sibl_spouse_aboard, parent_child_aboard, fare)
    VALUES """ + str(person[1:])
    pg_curs.execute(insert_person)

In [217]:
pg_conn.commit()

In [218]:
pg_curs = pg_conn.cursor()

pg_curs.execute('SELECT * FROM passengers_titanic LIMIT 20;')
pg_test = pg_curs.fetchall()

In [220]:
pg_test[0]

(1, 'no', 'third', 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25)